[Looking for PKS primers in iPKSes](#intro)  
[General working setup](#workEnv)  
[Getting the data](#gettingData)  
[Primer sequences](#primerSequences)  
[Confirm presence of primers](#confirm)  
[Retrieving sequences around primer matches](#retrievingFrags)  
[Alignment of primer-region fragments](#align)

<a id='intro'></a>
## Looking for PKS primers in iPKSes

We have a set of primers intended to targe polyketide synthases, from [Charlop-Powers (2014)](http://www.pnas.org/content/111/10/3757.abstract). These primers target the ketosynthase (KS) domain of Type I PKSes. 

The assumption, I think, is that these primers target bacterial gene clusters. Fungal Type I PKSes have traditionally been considered fundamentally different from bacterial Type I PKSes, for several reasons, mainly that they are iterative, and bacterial iterative Type I PPKSes were not thought to be very common.

[Charlop-Powers (2014)](http://www.pnas.org/content/111/10/3757.abstract) and other papers make it clear that these primers work on many bacterial type I PKSes, and therefore presumably non-iterative Type I PKSes. But since I am ostensibly some sort of mycologist, I cannot exclude fungi from any environmental survey. Do these primers work on fungal iterative PKSes? Additionally, it turns out that iterative Type PKSes are not necessarily uncommon, as per the paper we are going to dive into here, [Chen and Du (2016)](https://doi.org/10.1007/s00253-015-7093-0).

We'll use Chen and Du (2016) to check on a couple iPKSes, both fungal and bacterial, and begin to look at the behavior of these primers in iPKS systems. 

<a id='workEnv'></a>
### General working setup:

In [4]:
import pandas as pd
import shapely as sh
import numpy as np
import os, re, copy
from Bio import Entrez
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
Entrez.email = "danchurchthomas@gmail.com"

Imports for the sequence viewer. I can't find any widely-available plotter software that works well in notebooks, so this is a bokeh-based alignment viewer, copied from [dmnFarrell's great repo](https://dmnfarrell.github.io/bioinformatics/bokeh-sequence-aligner).

In [6]:
import os, io, random
import string
import numpy as np
import panel as pn
import panel.widgets as pnw
pn.extension()
import jupyter_seqPlotters
## homemade module, code stolen from 
## https://dmnfarrell.github.io/bioinformatics/bokeh-sequence-aligner

<a id='gettingData'></a>
### Getting the data

[Chen and Du (2016)](https://doi.org/10.1007/s00253-015-7093-0) is a really neat study showing that bacteria also use iPKSes (not just fungi), and that they are closely related proteins. They mostly cover bacterial PKSes, but they analyze a few fungal gene clusters, so we'll stand on their shoulders for this, and look at the sequences they used. They focused on the KS domain of the PKSes, which also happens to be the site for our candidate primers used by Charlop-Powers (2014). This is fortuitous, and why we will focus on this paper. 

This is some tabular data from Chen and Du (2016), that I transcribed from their phylogenetic analysis of some KS domains of some iPKSes: 

In [6]:
chenDu = pd.read_csv('ChenDuAccessions.csv', header=None)
chenDu.columns = ['name', 'id','NorP','bactOrFungi','nucID']
chenDu.head()

,name,id,NorP,bactOrFungi,nucID
0,AziB,ABY83164.1,p,b,EU240558.1
1,NcsB,AAM77986.1,p,b,AY117439.1
2,SACE5532,WP_009944722,p,b,NZ_PDBV01000001.1
3,PKS,EF028635.2,n,b,EF028635.2
4,PKS,KF954512.1,n,b,KF954512.1


  - `id` is the accession that Chen and Du list, though I had to update a few of these.  
  - `NorP`indicates whether the original id was a protein or nucleotide.  
  - `bactOrFungi` indicates whether the organism is bacterial or fungal.  
  - `nucID` shows the nucleotide accession that I used, which in the case of proteins was usually found with `Entrez` functions. 

We want to work with nucleotide sequences, not protein sequences, since we are testing primers. Most of Chen and Du's accession numbers are actually for proteins. But for the accessions that are nucleotides, we can grab these pretty easily from genbank:

In [9]:
nucs = chenDu[chenDu['NorP'] == 'n'].id.to_list()
for i in nucs:
    print(i)
    bb = Entrez.efetch(
        db="nuccore",
        rettype="fasta",
        retmode="text",
        id=i
    )
    record = SeqIO.read( bb, "fasta" )
    SeqIO.write(record, handle=record.id+'.fa', format='fasta')

EF028635.2
KF954512.1
ABYC01000481
KU597647.1


For finding associated nucleotide sequences from cases where Chen and Du list a protein sequence:

In [7]:
# our proteins are here:
prots = chenDu[chenDu['NorP'] == 'p'].id.to_list()
## remove problematic protein sequence:
prots.remove('WP_011873136.1')

## get ChenDu nucleotide sequences that are listed by prot accessions:
for i in prots:
    print(i)
    look4nuc = Entrez.elink(dbfrom='protein', db='nucleotide', id=i)
    aa = Entrez.read(look4nuc)
    idd = aa[0]['LinkSetDb'][0]['Link'][0]['Id']
    bb = Entrez.efetch(
        db="nuccore",
        rettype="fasta",
        retmode="text",
        id=idd
    )
    record = SeqIO.read( bb, "fasta" )
    print(i+" is "+ record.id)
    SeqIO.write(record, handle=record.id+'.fa', format='fasta')

ABY83164.1
ABY83164.1 is EU240558.1
AAM77986.1
AAM77986.1 is AY117439.1
WP_009944722
WP_009944722 is NZ_PDBV01000001.1
CAC20931.1
CAC20931.1 is AJ278573.1
AAO65796.1
AAO65796.1 is AF440781.1
CAE02602.1
CAE02602.1 is AJ575648.1
AAF26921.1
AAF26921.1 is AF210843.1
CAD19092.1
CAD19092.1 is AJ421825.1
AAD43562.2
AAD43562.2 is AF155773.5
AAD34559.1
AAD34559.1 is AH007774.2
ABA02240.1
ABA02240.1 is DQ176595.1
AAS90093.1
AAS90093.1 is AY510451.1
BAA18956.1
BAA18956.1 is D83643.1
AAN59953.1
AAN59953.1 is AF549411.1
AAQ17110.2
AAQ17110.2 is AY271660.2
AAM78012.1
AAM78012.1 is AY117439.1
AAL01060.1
AAL01060.1 is AF409100.1
BAA89382.2
BAA89382.2 is AB025342.2
AAK72879.2
AAK72879.2 is AF378327.2


I'm storing all the above fastas in a child directory called `./ChenDuAccs`.

<a id='primerSequences'></a>
### Primer sequences

We have the following sequences from Charlop-Powers (2014):

**KS2F** `GCIATGGAYCCICARCARMGIVT`  
**KS2R** `GTICCIGTICCRTGISCYTCIAC`

We need to get these into a format that works for searching our sequences. For the ambiguous nucleotides and inosine, a good wikipedia reference is [here](https://en.wikipedia.org/wiki/Nucleotide#Abbreviation_codes_for_degenerate_bases). 'I' is for inosine, pairs with A, C, or T so, I = [TGA]? Not sure, try it.


In [16]:
###############################################################
##    GC  I  ATGGA Y  CC  I  CA R  CA R   M  G  I   V   T
KS2F="GC[TGA]ATGGA[CT]CC[TGA]CA[AG]CA[AG][AC]G[TGA][ACG]T"
###############################################################

###############################################################
## reverse primer
##      GT  I  CC  I  GT  I  CC R  TG  I   S  C Y  TC  I  AC
KS2R = "GT[TGA]CC[TGA]GT[TGA]CC[AG]TG[TGA][CG]C[CT]TC[TGA]AC"
###############################################################

###############################################################
##       'A]CGT[]TCA[C]GT[]CT[TG]CT[TG]TCA[GG]AG[TCCAT]TCA[GC'
KS2Frc = 'A[CGT][TCA]C[GT][CT]TG[CT]TG[TCA]GG[AG]TCCAT[TCA]GC'
###############################################################

###############################################################
##       'GT]TCA[GA]AG[G]CG[]TCA[CA]CT[GG]TCA[AC]TCA[GG]TCA[AC'
KS2Rrc = 'GT[TCA]GA[AG]G[CG][TCA]CA[CT]GG[TCA]AC[TCA]GG[TCA]AC'
###############################################################


To sum up:

In [19]:
KS2F = "GC[TGA]ATGGA[CT]CC[TGA]CA[AG]CA[AG][AC]G[TGA][ACG]T"
KS2R = "GT[TGA]CC[TGA]GT[TGA]CC[AG]TG[TGA][CG]C[CT]TC[TGA]AC"
KS2Frc = 'A[CGT][TCA]C[GT][CT]TG[CT]TG[TCA]GG[AG]TCCAT[TCA]GC'
KS2Rrc = 'GT[TCA]GA[AG]G[CG][TCA]CA[CT]GG[TCA]AC[TCA]GG[TCA]AC'

Compile the regex expressions:

In [18]:
KS2Fregex = re.compile(KS2F)
KS2Rregex = re.compile(KS2R)
KS2Frcregex = re.compile(KS2Frc)
KS2Rrcregex = re.compile(KS2Rrc)

<a id='confirm'></a>
### Confirm presence of primers

Do any of these iPKS KS sequences actually contain exact matches to our primers? If so, these primers may be useful.  

In [22]:
hits = []
desc = []
for i in os.listdir('ChenDuAccs'):
    aa = SeqIO.parse(open(('ChenDuAccs/'+i), mode='r'), 'fasta')
    rec = list(aa)[0]
    if KS2Fregex.search(str(rec.seq)): hits.append(i); desc.append(rec.description)
    if KS2Rregex.search(str(rec.seq)): hits.append(i); desc.append(rec.description)
    if KS2Frcregex.search(str(rec.seq)): hits.append(i); desc.append(rec.description)
    if KS2Rrcregex.search(str(rec.seq)): hits.append(i); desc.append(rec.description)
    hits = [ i.replace('.fa', '') for i in hits ]
    hitsU = list(set(hits))

Sequences from Chen and Du (2016) that contain an exact match to least one of our primer sites:

In [24]:
hitsU

['NZ_PDBV01000001.1',
 'AF155773.5',
 'AF440781.1',
 'AF378327.2',
 'AJ575648.1',
 'AJ278573.1']

In [26]:
hasPrim = chenDu[chenDu['nucID'].isin(hitsU)]
hasPrim.reset_index(inplace=True, drop=True)
hasPrim

,name,id,NorP,bactOrFungi,nucID
0,SACE5532,WP_009944722,p,b,NZ_PDBV01000001.1
1,PimS1,CAC20931.1,p,b,AJ278573.1
2,MonAI,AAO65796.1,p,b,AF440781.1
3,AurA,CAE02602.1,p,b,AJ575648.1
4,Fum1p,AAD43562.2,p,f,AF155773.5
5,OrfA,AAK72879.2,p,b,AF378327.2


6 out of 19 accessions had exact matches in them. There is one example of a fungal sequence with this primer site, the Fumonisin biosynthetic gene cluster Fum1p. The primers as they are pick up a significant portion of the iPKSes, including a fungal PKS, though not all iPKSes had exact matches. This is promising, because we haven't tweaked the primers in any way to include iPKSes. 

<a id='retrievingFrags'></a>
### Retrieving sequences around primer matches

Charlop-Powers (2014) reports that these primers yielded sequences with lengths in the range of 350-500 base pairs. We are going to grab 500 bp fragments from the sequences of these gene clusters from around these primer sites that we are detecting. 

Define a few functions to do this:

In [28]:
def getPrimerSpan(fafile, primer):
    """use one of the compiled regexes above on a fasta
    file to get locations of the primer sites"""
    with open((fafile), mode='r') as f:
        aa = SeqIO.parse(f, 'fasta')
        sequ = str(list(aa)[0].seq)
    primiter = primer.finditer(sequ)
    if primiter:
        primerSpans = [ i.span() for i in list(primiter) ]
    else:
        primerSpans = None
        print('primer not found')
    return(primerSpans)


In [29]:
def getBPfromSite(fastafile, site, bp):
    """function to get sequence chunks from before or past a primer site."""
    with open(fastafile, mode='r') as f:
        seqIter = SeqIO.parse(f, 'fasta')
        sequ = list(seqIter)[0].seq
    if bp < 0:
        seqFrag = sequ[(site[1]+bp):site[1]]
    elif bp > 0:
        seqFrag = sequ[site[0]:(site[1]+bp)]
    elif bp == 0:
        seqFrag = None
    return(seqFrag)

In [30]:
def getSeqsForAPrimer(fastafile, primer, seqName, seqLength):
    """use getPrimerSpan() and getBPfromSite() to collect the sequence
        fragments from each matching site to a primer and put into a 
        list of biopython sequence record objects"""
    sites = getPrimerSpan(fastafile,primer)
    seqs=[]
    for sitenumber, site in enumerate(sites):
        sequence = getBPfromSite(hasPrimerFiles[0], site=site, bp=seqLength)
        seqid = seqName+'_S'+str(sitenumber)
        seqRec = SeqRecord(sequence)
        seqRec.id = seqid
        seqs.append(seqRec)
    return(seqs)

We're going to iterate over both our fastas that contain primer hits, and over all four primer regexes (F,R,Frc,Rrc).

In [31]:
# make a series of primers to iterate over
primerRegexes = [KS2Fregex, KS2Rregex, KS2Frcregex, KS2Rrcregex]
primerRegexesNames = ['KS2F', 'KS2R', 'KS2Frc', 'KS2Rrc']
primSeries = pd.Series(primerRegexes, index=primerRegexesNames)

## files to iterate:
hasPrimerFiles = [ ("ChenDuAccs/"+i+".fa") for i in hasPrim.nucID.values ]

Make a list of sequence records and collect them in a fasta file. Each primer site from each original accesion/sequence will receive its own entry in the new fasta file. There are often multiple hits for each primer in a given accession/sequence.

In [37]:
accessionseqs=[]
for fileNu, file in enumerate(hasPrimerFiles):
    print(file)
    for primNu, prim in enumerate(primSeries):
        seqName=(hasPrim.nucID.values[fileNu] + '_primer-' +primSeries.index[primNu])
        #print(seqName)
        if 'F' in primSeries.index[primNu]: n = (+1)
        elif 'R' in primSeries.index[primNu]: n = (-1)
        primHitSeqs=getSeqsForAPrimer(file, primer=prim, seqName=seqName, seqLength=(500*n))
        accessionseqs += primHitSeqs

## write out
SeqIO.write(accessionseqs, 'chenDuPrimerHits.fa', 'fasta')

ChenDuAccs/NZ_PDBV01000001.1.fa
ChenDuAccs/AJ278573.1.fa
ChenDuAccs/AF440781.1.fa
ChenDuAccs/AJ575648.1.fa
ChenDuAccs/AF155773.5.fa
ChenDuAccs/AF378327.2.fa


19

<a id='align'></a>
### Alignment of primer-region fragments

And time to do an alignment and see if these results make any sense. In `BASH`, using [MUSCLE](https://www.drive5.com/muscle/manual/). 

In [2]:
muscle -in chenDuPrimerHits.fa -out testAlignChenduHits.fa


MUSCLE v3.8.31 by Robert C. Edgar

http://www.drive5.com/muscle
This software is donated to the public domain.
Please cite: Edgar, R.C. Nucleic Acids Res 32(5), 1792-97.

chenDuPrimerHits 19 seqs, max length 523, avg  length 507
00:00:00    10 MB(-2%)  Iter   1  100.00%  K-mer dist pass 1
00:00:00    10 MB(-2%)  Iter   1  100.00%  K-mer dist pass 2
00:00:00    19 MB(-4%)  Iter   1  100.00%  Align node       
00:00:00    19 MB(-4%)  Iter   1  100.00%  Root alignment
00:00:00    19 MB(-4%)  Iter   2  100.00%  Refine tree   
00:00:00    19 MB(-4%)  Iter   2  100.00%  Root alignment
00:00:00    19 MB(-4%)  Iter   2  100.00%  Root alignment
00:00:01    19 MB(-4%)  Iter   3  100.00%  Refine biparts
00:00:01    19 MB(-4%)  Iter   4  100.00%  Refine biparts
00:00:02    19 MB(-4%)  Iter   5  100.00%  Refine biparts
00:00:02    19 MB(-4%)  Iter   6  100.00%  Refine biparts
00:00:02    19 MB(-4%)  Iter   7  100.00%  Refine biparts
00:00:02    19 MB(-4%)  Iter   7  100.00%  Refine biparts


And back into python, see if we can view this alignment in our notebook:

In [11]:

p = jupyter_seqPlotters.view_alignment(aln, plot_width=900)
pn.pane.Bokeh(p)

Bokeh(Column)

## Plants, PKS and NRPS primers

As a general indicator, how many records do we find in the NCBI nucleotide database for PKSes, type 1?

In [3]:
plantPKShandle = Entrez.esearch(db="nucleotide",
    term="(((Polyketide Synthase) AND Type I) NOT Type II) NOT Type III AND green plants[porgn:__txid33090]",
    idtype="acc",
    retmax=1000,
    usehistory="y",
)


In [5]:
record = Entrez.read(plantPKShandle)
webenv = record["WebEnv"]
query_key = record["QueryKey"]

In [7]:
record['Count']

'246'

So in the vast nucleotide database, we find 246 references to PKS I in plants...

In [9]:
fetch = Entrez.efetch(
    db="nucleotide",
    rettype="fasta",
    retmode="text",
    retmax=len(record["IdList"]),
    webenv=webenv,
    query_key=query_key,
    idtype="acc",
)

In [10]:
dataPlantPKS = fetch.read()

In [11]:
pksfile='/home/daniel/Documents/job_apps/panama/bigFilesBiodiv/plantPKS.fa'
with open(pksfile, 'w') as f:
    f.write(dataPlantPKS)
    fetch.close()

In [14]:
aln = AlignIO.read('testCombo.afa','fasta')
p = view_alignment(aln, plot_width=900)
pn.pane.Bokeh(p)


Bokeh(Column)